In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [21]:
# Load the trained model, scaler, onehot
model=load_model("model.h5")

with open("onehot_encoder_geo.pkl","rb") as file:
    label_encoder_geo=pickle.load(file)

with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender=pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

In [43]:
# Exeample input data
input_data = {
    "CreditScore" : 600,
    "Geography" : "France",
    "Gender" : "Male",
    "Age" : 40,
    "Tenure" : 3,
    "Balance" : 60000,
    "NumOfProducts" : 2,
    "HasCrCard" : 1,
    "IsActiveMember" : 1,
    "EstimatedSalary" : 50000
}

In [44]:
df = pd.DataFrame([input_data])

In [45]:
# Onehot encode Geography
geo_encoded = label_encoder_geo.transform(df[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [46]:
df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [47]:
# Label encoder Gender
df["Gender"] = label_encoder_gender.transform(df["Gender"])

In [48]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [49]:
# Scaler
df = scaler.transform(df)

In [50]:
df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [51]:
# Prediction
prediction=model.predict(df)

1/1 [==============================] - 0s 122ms/step


In [52]:
prediction

array([[0.01831235]], dtype=float32)

In [55]:
prediction = prediction[0][0]
prediction

0.018312348

In [56]:
if prediction > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
